In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/MSc_project/hashtag-crawler

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/MSc_project/hashtag-crawler


In [7]:
# !pip install language_tool_python
# !pip install pycountry
# !pip install langdetect
# !pip install nltk
!pip install tweet-preprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from enum import unique
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import os.path
import nltk
import nltk.data
import time
import string

# nltk.download('vader_lexicon')
# nltk.download('punkt')
# nltk.download('stopwords')
import json

import language_tool_python
import preprocessor as p

import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from nltk import word_tokenize
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

class Analyzer(object):
    def __init__(self, hashtag):

        self.hashtag = hashtag
        self.save_path = f'informer_results{os.path.sep}{hashtag}'

        self.tool = language_tool_python.LanguageTool('en-US')
        self.alphabets= "([A-Za-z])"
        self.prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
        self.suffixes = "(Inc|Ltd|Jr|Sr|Co)"
        self.starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
        self.acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
        self.websites = "[.](com|net|org|io|gov)"
        # self.informer_db = informer_db

    def load_informer_data(self):
        path = f'{self.hashtag}{os.path.sep}informer_database.json'
        with open(path) as jf:
            data = json.load(jf)
        return data

    @staticmethod
    def get_the_tweets(database):
        all_tweets = {}
        for key,value in database.items():
            all_tweets.update( {key:value['tweet-text']})
            for informer in value['informers-data']:
                all_tweets.update({informer['id']:informer['tweet-text']})
        return all_tweets

    @staticmethod
    def store_by_tweets(database):
        all_tweets = {}
        for key,value in database.items():
            if value in all_tweets:
                new = all_tweets[value].append(key)
                all_tweets[value] = new
            else:
                all_tweets[value] = [key]

        return all_tweets

        
    def tweet_cleaner(self,tw_list):
        remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
        rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
        hash = lambda x: re.sub(r'#', "", x)
        amp = lambda x: re.sub(r'&amp', "", x)


        tw_list['grammartext'] = tw_list.text.map(remove_rt).map(rt)
        tw_list['clean_text'] = tw_list.text.map(remove_rt).map(rt)
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.HASHTAG)
        tw_list["grammartext"] = tw_list.grammartext.map(p.clean)
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION)
        tw_list["clean_text"] = tw_list.clean_text.map(p.clean).map(hash).map(amp)
        tw_list["clean_text"] = tw_list.clean_text.str.lower()

        #Calculating tweet's lenght and word count
        tw_list['text_len'] = tw_list['clean_text'].astype(str).apply(len)
        tw_list['text_word_count'] = tw_list['clean_text'].apply(lambda x: len(str(x).split()))
        tw_list['punct'] = tw_list['clean_text'].apply(lambda x: self.remove_punct(x))
        tw_list['tokenized'] = tw_list['punct'].apply(lambda x: self.tokenization(x.lower()))
        tw_list['nonstop'] = tw_list['tokenized'].apply(lambda x: self.remove_stopwords(x))
        tw_list['stemmed'] = tw_list['nonstop'].apply(lambda x: self.stemming(x))
        return tw_list


    def get_sentiment(self,index,row):
        score = SentimentIntensityAnalyzer().polarity_scores(row.clean_text)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']
        comp = score['compound']
        if neg > pos:
            self.df.loc[index, 'sentiment'] = "negative"
        elif pos > neg:
            self.df.loc[index, 'sentiment'] = "positive"
        else:
            self.df.loc[index, 'sentiment'] = "neutral"
        self.df.loc[index, 'neg'] = neg
        self.df.loc[index, 'neu'] = neu
        self.df.loc[index, 'pos'] = pos
        self.df.loc[index, 'compound'] = comp


    def get_grammar(self,index, row):
        # https://michaeljanz-data.science/deepllearning/natural-language-processing/scoring-texts-by-their-grammar-in-python/
        scores_word_based_sentence = []
        scores_sentence_based_sentence = []
        s1 = time.perf_counter()
        sentences = nltk.tokenize.sent_tokenize(row.grammartext)
        e1 = time.perf_counter()
        # print(f'tokenizer took {e1-s1}s to complete')
        # print(sentences)
        # sentences = self.split_into_sentences(row)
        for sentence in sentences:
        # for sentence in helpers.text_to_sentences(text):
            s2 = time.perf_counter()
            matches = self.tool.check(sentence)
            e2 = time.perf_counter()
            # print(f'language tool took {e2-s2}s ')
            count_errors = len(matches)
            # only check if the sentence is correct or not
            scores_sentence_based_sentence.append(np.min([count_errors, 1]))
            scores_word_based_sentence.append(count_errors)
            
        word_count = len(nltk.tokenize.word_tokenize(row.grammartext))
        sum_count_errors_word_based = np.sum(scores_word_based_sentence)
        score_word_based = 1 - (sum_count_errors_word_based / word_count)
        
        sentence_count = len(sentences)       
        sum_count_errors_sentence_based = np.sum(scores_sentence_based_sentence)
        score_sentence_based = 1 - np.sum(sum_count_errors_sentence_based / sentence_count)

        self.df.loc[index, 'grammar-word-scoure'] = score_word_based
        self.df.loc[index, 'grammar-sentence-score'] = score_sentence_based


    @staticmethod
    def count_values_in_column(data,feature):
        total=data.loc[:,feature].value_counts(dropna=False)
        percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
        return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])


    def make_sentiment_pi_chart(self,tw_list):
        # create data for Pie Chart
        pichart = self.count_values_in_column(tw_list,"sentiment")
        names= pichart.index
        size=pichart["Percentage"]
        
        # Create a circle for the center of the plot
        my_circle=plt.Circle( (0,0), 0.7, color='white')
        plt.pie(size, labels=names, colors=['green','blue','red'])
        p=plt.gcf()
        p.gca().add_artist(my_circle)
        # plt.show()
        plt.savefig(f'{self.save_path}_sentiment_summarisation_pichart.png')

    def create_wordcloud(self,text,label):
        path = f'{self.save_path}_wordcloud_{label}.png'
        stopwords = set(STOPWORDS)
        wordcloud = WordCloud(width=1600, height=800,max_font_size=200,max_words=1000,stopwords=stopwords, background_color='white', repeat=False).generate(str(text))
        plt.figure(figsize=(12,10))
        plt.imshow(wordcloud, interpolation="bilinear")
        plt.axis("off")
        plt.savefig(path)
        # plt.show()


    def get_wordclouds(self,tw_list):
        
        tw_list_negative = tw_list[tw_list["sentiment"]=="negative"]['clean_text']
        tw_list_positive = tw_list[tw_list["sentiment"]=="positive"]['clean_text']
        tw_list_neutral = tw_list[tw_list["sentiment"]=="neutral"]['clean_text']

        #Creating wordcloud for all tweets
        self.create_wordcloud(tw_list["clean_text"].values,label='all')

        #Creating wordcloud for positive sentiment
        self.create_wordcloud(tw_list_positive.values,label ='positive')

        #Creating wordcloud for negative sentiment
        self.create_wordcloud(tw_list_negative.values,label = 'negative')

        #Creating wordcloud for neutral sentiment
        self.create_wordcloud(tw_list_neutral.values,label = 'neutral')

    def remove_punct(self,text):
        text = "".join([char for char in text if char not in string.punctuation])
        text = re.sub('[0–9]+', '', text)
        text = re.sub( "'", "",text)
        text = re.sub( '"', '',text)

        return text


    def remove_stopwords(self,text):
        self.stopword = nltk.corpus.stopwords.words('english')
        text = [word for word in text if word not in self.stopword]
        return text

    def stemming(self,text):
        self.ps = nltk.PorterStemmer()
        text = [self.ps.stem(word) for word in text]
        return text

    def clean_text(self,text):
        text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
        text_rc = re.sub('[0-9]+', '', text_lc)
        tokens = re.split('\W+', text_rc)    # tokenization
        text = [self.ps.stem(word) for word in tokens if word not in self.stopword]  # remove stopwords and stemming
        return text

    @staticmethod
    def tokenization(text):
        text = re.split('\W+', text)
        return text


    def get_density_of_words(self,tw_list):

        #Appliyng Countvectorizer
        countVectorizer = CountVectorizer(analyzer=self.clean_text) 
        countVector = countVectorizer.fit_transform(tw_list['stemmed'])
        print('{} Number of reviews has {} words'.format(countVector.shape[0], countVector.shape[1]))
        #print(countVectorizer.get_feature_names())
        count_vect_df = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())

        # Most Used Words
        count = pd.DataFrame(count_vect_df.sum())
        countdf = pd.DataFrame(count.sort_values(0,ascending=False))
        countdf.to_csv(f'{self.save_path}_most_used_words.csv')

    @staticmethod
    def sort_by_tweet(all_tweets): 

        df = pd.DataFrame.from_dict(all_tweets, orient='index', columns= ['text'])
        sorted_tweets = {}
        for row,index in df.groupby('text').groups.items():
            key = tuple(index.values.tolist())
            sorted_tweets.update({key:row})

        new_df = pd.DataFrame.from_dict(sorted_tweets, orient='index', columns= ['text'])
        
        return new_df


    def tweet_analysis(self):

        print(f'Performing analysis on {self.hashtag}')

        self.informer_db = self.load_informer_data()

        all_tweets = self.get_the_tweets(self.informer_db)
        self.df = pd.DataFrame.from_dict(all_tweets, orient='index', columns= ['text'])

        tweet_df = self.sort_by_tweet(all_tweets)

        tweet_df = self.tweet_cleaner(tweet_df)
        self.df = self.tweet_cleaner(self.df)

        self.df[['polarity', 'subjectivity']] = self.df['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

        # index the rows of the database that contains the unique tweets
        for index, row in tweet_df.iterrows():

            self.get_sentiment(index,row)
            self.get_grammar(index,row)

        print('finished getting the metrics!!!!!!!')

        df_path = f'{self.save_path}_tweets_classification.csv'
        self.df.to_csv(df_path)
        
        # refilter the database to only contain the unique tweets
        unique_df = self.df.copy()
        unique_df.drop_duplicates('clean_text', inplace=True)

        sentiment_count = self.count_values_in_column(unique_df,"sentiment")
        sentiment_count.to_csv(f'{self.save_path}_sentiment_summarisation.csv')

        # make a pi chart of the result of sentiments
        self.make_sentiment_pi_chart(unique_df)

        ### Create word clouds of all 
        self.get_wordclouds(unique_df)

        # get thedensity of the words used
        self.get_density_of_words(unique_df)


In [15]:
hashtags = ['avengers','blm','borisjohnson','brexit','climatechange','covid','gaza','loveisland','monkeypox','nhs','olivianewtonjohn','supercup','UkraineWar']
# hashtag = 'gaza'

informer_db_path = f'informer_results{os.path.sep}'

for hashtag in hashtags:

    a = Analyzer(hashtag)

    a.tweet_analysis()

Output hidden; open in https://colab.research.google.com to view.